In [1]:
from arcgis.gis import GIS
from arcgis.geocoding import geocode
from IPython.display import display
my_gis = GIS()

In [8]:

my_gis.map("Germany",6)

MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oceans', 'osm', 'satellite', 'streets', 'streets-navigation-vector', 'streets-night-vector', 'streets-relief-vector', 'streets-vector', 'terrain', 'topo', 'topo-vector'], center=[51.11063104900006, 10.39227793200007], zoom=6)

In [4]:
public_content = my_gis.content.search("Fire", item_type="Feature Layer", max_items=5)

In [18]:
for item in public_content:
    display(item)



<Item title:"Collier County Emergency Services" type:Feature Layer Collection owner:philsherman_collierbcc>

<Item title:"מיפוי שריפה חדש" type:Feature Layer Collection owner:kkl_arcgis_10>

<Item title:"CalOES 2017 Fire Points" type:Feature Layer Collection owner:calema_gis>

<Item title:"Fire Perimeters" type:Feature Layer Collection owner:cferner_CALFIRE>

<Item title:"Intact Forest Landscapes (2000)" type:Feature Layer Collection owner:GlobalForestWatch>

In [20]:
map1 = my_gis.map('Collier County, FL')
#get the first item
fire = public_content[0]

#add to map
map1.add_layer(fire)

map1

MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oceans', 'osm', 'satellite', 'streets', 'streets-navigation-vector', 'streets-night-vector', 'streets-relief-vector', 'streets-vector', 'terrain', 'topo', 'topo-vector'], center=[0, 0])

In [26]:
# set the map's extent by geocoding the location
diegogarcia = geocode("Diego Garcia")[0]
map1.extent = diegogarcia['extent']
# geocode location of La Reunion island
lareunion = geocode("La Reunion")[0]

In [27]:


# Annotate the map by plotting Diego Garcia, and two other search locations
map1.draw(lareunion['location'], {"title": "Reunion Island", "content": "Debris found"})
map1.draw(diegogarcia['location'], {"title": "Diego Garcia", "content": "Naval Support Facility Diego Garcia"})

# Render a feature layer representing the search area
# Source: http://www.amsa.gov.au/media/incidents/images/DIGO_00718_01_14.jpg
map1.add_layer({"type":"FeatureLayer", 
                "url" : "http://services.arcgis.com/WQ9KVmV6xGGMnCiQ/arcgis/rest/services/MH370Search/FeatureServer/1"})



In [2]:
import csv
import re
import math
import time
import random
import numpy as np
import sys
import os


from pyspark import SparkContext, SparkConf
from matplotlib.pyplot import axis
from numpy import shape

import copy
import pandas
import matplotlib
import datetime
import matplotlib as mpl
import pandas as pd

from sklearn.gaussian_process.kernels import Matern
import matplotlib.colors as colors
import matplotlib.pyplot as plt
#os.environ['SPARK_CLASSPATH'] = './spark-streaming-kafka-0-8-assembly_2.11-2.3.0.jar'
#os.environ['SPARK_CLASSPATH'] = './spark-sql-kafka-0-10_2.11-2.3.0.jar'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.3.0 pyspark-shell'
#    Spark
from pyspark import SparkContext
#    Spark Streaming
from pyspark.streaming import StreamingContext  
#    Kafka
from pyspark.streaming.kafka import KafkaUtils  
#    json parsing
import json 


from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [3]:
conf = SparkConf().setMaster("local[*]").setAppName('pyspark')#.set("spark.jars", "./spark-sql-kafka-0-10_2.11-2.3.0.jar")
#ss = SparkSession.builder.config(conf=conf).getOrCreate()
ss = SparkSession.builder.config(conf=conf).getOrCreate()

In [4]:
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import *
'''
def strAsDict(dictStr):
    import ast
    return ast.literal_eval(dictStr)
myAsDict = udf(strAsDict,MapType(ArrayType()))'''


#Create a DataFrame that streams data from Kafka
df = ss.readStream.format("kafka").option("kafka.bootstrap.servers", "smartaqnet-dev01.teco.edu:9092") \
        .option("subscribe", "WG_Measurements").load()

In [5]:
df = df.selectExpr("CAST(timestamp AS STRING)", "CAST(value AS STRING)")
mySchema = StructType([StructField("AirTemperature", FloatType()),
                       StructField("Latitude", FloatType()),
                       StructField("Longitude", FloatType()),
                       StructField("ID", StringType()),
                      StructField("Time", StringType())])
df = df.select(F.regexp_replace('value', "u\'", "\'").alias('value'))
df = df.select(F.regexp_replace('value', "\'", "\"").alias('value'))
df = df.select(df.value,F.get_json_object(df.value, '$.ID').alias("ID"))
df = df.groupBy(df.ID).agg(F.last(df.value).alias("value"))
df = df.select(F.get_json_object(df.value, '$.ID').alias("ID"),
               F.get_json_object(df.value, '$.Time').alias("Time"),
               F.get_json_object(df.value, '$.Latitude').alias("Latitude"),
               F.get_json_object(df.value, '$.Longitude').alias("Longitude"),
               F.get_json_object(df.value, '$.AirTemperature').alias("AirTemperature"))

In [6]:
q = df.writeStream.queryName('TemperatureMap').outputMode("complete").format("memory").start()

In [9]:
map1 = my_gis.map("Germany",6)
#map1.add_layer({"type":"SensorLayer","url":"NothingUrl"})

                
#Continuesly get range
while(True):
    
    columns  = ['ID',
         'Time',
         'Latitude',
         'Longitude',
         'AirTemperature'
        ]
    currentState = pd.DataFrame(ss.sql("select * from TemperatureMap").rdd.collect(),columns =columns )
    currentState[['Latitude','Longitude','AirTemperature']] = currentState[['Latitude','Longitude','AirTemperature']].apply(pd.to_numeric,errors="coerce")
    knownPoints = currentState.dropna(axis=0, how='any').values[:,:]#.astype(np.float)
    print(knownPoints.shape[0])
    #print "{0}: {1},\t{2}: {3}".format(knownPoints[:,0].min(), knownPoints[:,0].max(),knownPoints[:,1].min(), knownPoints[:,1].max())
    for aRow in knownPoints:
        map1.draw([aRow[2], aRow[3]], {"title":"ID", "content":aRow[0]})
    map1.getLayer("SensorLayer").refresh()
    time.sleep(30)


2680


AttributeError: 'MapView' object has no attribute 'getLayer'

In [ ]:
from arcgis.geometry import Point
columns  = ['ID',
     'Time',
     'Latitude',
     'Longitude',
     'AirTemperature'
    ]
currentState = pd.DataFrame(ss.sql("select * from TemperatureMap").rdd.collect(),columns =columns )
currentState[['Latitude','Longitude','AirTemperature']] = currentState[['Latitude','Longitude','AirTemperature']].apply(pd.to_numeric,errors="coerce")
knownPoints = currentState.dropna(axis=0, how='any').loc[0:500,:]
fc = my_gis.content.import_data(knownPoints, {"x":"Latitude","y":"Longitude","Renderer":"SimpleRenderer"})
map1 = my_gis.map("Germany",5)
map1.add_layer(fc)
map1

In [91]:
columns  = ['ID',
     'Time',
     'Latitude',
     'Longitude',
     'AirTemperature'
    ]
currentState = pd.DataFrame(ss.sql("select * from TemperatureMap").rdd.collect(),columns =columns )
currentState[['Latitude','Longitude','AirTemperature']] = currentState[['Latitude','Longitude','AirTemperature']].apply(pd.to_numeric,errors="coerce")
knownPoints = currentState.dropna(axis=0, how='any').values[:,:]#.astype(np.float)
print(knownPoints.shape[0])
fro,too = -10.0,30.0
mySymbol = {
"type": "esriSMS",
 "style": "esriSMSSquare",
 "color": [76,115,0,255],
 "size": 8,
 "angle": 0,
 "xoffset": 0,
 "yoffset": 0,
 "outline": 
  {
  "color": [152,230,0,255],
   "width": 1
  }
}
for aRow in knownPoints[:50]:
    map1.draw([aRow[2], aRow[3]], symbol = mySymbol)

2895


In [97]:
map1.get_state()

{'_addlayer': '',
 '_arcgis_url': '',
 '_basemap': 'topo',
 '_dom_classes': (),
 '_extent': '',
 '_gallerybasemaps': [],
 '_gbasemaps_def': [],
 '_js_basemap': '[{"url":"http://services.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer","title":"World Topographic Map"}]',
 '_js_interactive_drawn_graphic': '',
 '_js_layer_list': '',
 '_js_renderer': '',
 '_jsextent': '{"type":"extent","xmin":-2296956.1759882607,"ymin":5484218.777761568,"xmax":2482498.3286259696,"ymax":7441006.701861559,"spatialReference":{"wkid":102100,"latestWkid":3857}}',
 '_layerId_to_remove': '',
 '_model_module': '@jupyter-widgets/base',
 '_model_module_version': '1.0.0',
 '_model_name': 'DOMWidgetModel',
 '_swipe_div': 'swipeDiv5ITLN3',
 '_token_info': '',
 '_view_count': None,
 '_view_module': 'mapview',
 '_view_module_version': '',
 '_view_name': 'MapView',
 'basemaps': ['dark-gray',
  'dark-gray-vector',
  'gray',
  'gray-vector',
  'hybrid',
  'national-geographic',
  'oceans',
  'osm',
  'satelli

In [87]:
map1 = my_gis.map("Karlsruhe",7)


MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oceans', 'osm', 'satellite', 'streets', 'streets-navigation-vector', 'streets-night-vector', 'streets-relief-vector', 'streets-vector', 'terrain', 'topo', 'topo-vector'], center=[49.01094000000006, 8.408450000000073], zoom=7)

In [51]:
aRow=knownPoints[101]
map1.draw([aRow[2], aRow[3]], {"title":"ID", "content":aRow[0]})

In [45]:
fc.properties

{
  "layerDefinition": {
    "currentVersion": 10.51,
    "id": 0,
    "name": "",
    "type": "Feature Layer",
    "displayField": "",
    "description": "",
    "copyrightText": "",
    "defaultVisibility": true,
    "relationships": [],
    "isDataVersioned": false,
    "supportsAppend": true,
    "supportsCalculate": true,
    "supportsTruncate": false,
    "supportsAttachmentsByUploadId": true,
    "supportsAttachmentsResizing": true,
    "supportsRollbackOnFailureParameter": true,
    "supportsStatistics": true,
    "supportsAdvancedQueries": true,
    "supportsValidateSql": true,
    "supportsCoordinatesQuantization": true,
    "supportsApplyEditsWithGlobalIds": false,
    "advancedQueryCapabilities": {
      "supportsPagination": true,
      "supportsPaginationOnAggregatedQueries": true,
      "supportsQueryRelatedPagination": true,
      "supportsQueryWithDistance": true,
      "supportsReturningQueryExtent": true,
      "supportsStatistics": true,
      "supportsOrderBy": tru

In [48]:
from arcgis.features import SpatialDataFrame
sdf = SpatialDataFrame.from_df(df)

ValueError: Address column not found in dataframe

In [20]:
map1

MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oceans', 'osm', 'satellite', 'streets', 'streets-navigation-vector', 'streets-night-vector', 'streets-relief-vector', 'streets-vector', 'terrain', 'topo', 'topo-vector'], center=[51.11063104900006, 10.39227793200007], mode='{"geometry": {"x": 11.790999999999999, "y": 50.286, "spatialReference": {"wkid": 4326}, "type": "point"}, "infoTemplate": {"title": "ID", "content": "IBAVARIA14"}, "symbol": null, "attributes": null}', zoom=6)